## State data pipeline

Here we define state data to be the complementary data of the magnetic field data, useful for the analysis of the magnetic field data. It may include:

- Plasma data
- Satellite location data

### Loading data

In [ ]:
def download_state_data(
    start: str = None,
    end: str = None,
    ts: str = None,  # time resolution
    probe: str = None,
    coord: str = None,
):
    ...


def load_state_data(
    start: str = None,
    end: str = None,
    ts: str = None,  # time resolution
    probe: str = None,
    coord: str = None,
):
    """Loading the raw dataset
    
    - Downloading data
    - Reading data into a proper data structure, like dataframe.
        - Parsing original data (dealing with delimiters, missing values, etc.)
    """
    ...


### Preprocessing data

In [ ]:
def preprocess_state_data(
    raw_data: Any = None,
    start: str = None,
    end: str = None,
    ts: str = None,  # time resolution
    coord: str = None,
) -> pl.DataFrame:
    """
    Preprocess the raw dataset (only minor transformations)

    - Applying naming conventions for columns
    - Parsing and typing data (like from string to datetime for time columns)
    - Structuring the data (like pivoting, unpivoting, etc.)
    - Changing storing format (like from `csv` to `parquet`)
    - Dropping null columns
    - Resampling data to a given time resolution (better to do in the next stage)
    - ... other 'transformations' commonly performed at this stage.
    """
    pass


### Processing data

In [ ]:
def process_state_data(df: pl.DataFrame, columns=None,) -> pl.DataFrame:
    """
    Corresponding to primary data layer, where source data models are transformed into domain data models

    - Transforming data to RTN (Radial-Tangential-Normal) coordinate system
    - Discarding unnecessary columns
    - Smoothing data
    - Resampling data to a given time resolution
    - Partitioning data, for the sake of memory
    """
    pass

### Pipeline

In [ ]:
def create_state_data_pipeline(
    sat_id,
    ts: str = '1h',  # time resolution
) -> Pipeline:
    
    node_load_data = node(
        load_state_data,
        inputs=dict(
            start="params:start_date",
            end="params:end_date",
        ),
        outputs="raw_state",
        name=f"download_{sat_id.upper()}_state_data",
    )
    
    node_preprocess_data = node(
        preprocess_state_data,
        inputs=dict(
            raw_data="raw_state",
            start="params:start_date",
            end="params:end_date",
        ),
        outputs=f"inter_state_{ts}",
        name=f"preprocess_{sat_id.upper()}_state_data",
    )
    
    node_process_data = node(
        process_state_data,
        inputs=f"inter_state_{ts}",
        outputs=f"primary_state_{ts}",
        name=f"process_{sat_id.upper()}_state_data",
    )

    nodes = [
        node_load_data,
        node_preprocess_data,
        node_process_data,
    ]

    pipelines = pipeline(
        nodes,
        namespace=sat_id,
        parameters={
            "params:start_date": "params:jno_start_date",
            "params:end_date": "params:jno_end_date",
        },
    )

    return pipelines